In [1]:
import matplotlib.pyplot as plt
import numpy as np
import sympy as sy
from scipy.special import sph_harm 
from scipy import integrate
from scipy.integrate import dblquad
from scipy.integrate import quad
%run -i func_proj_reconst.ipynb

Input Streamfunction:

$$
\psi(\theta, \phi)= Re(Y_n^m(\theta,\phi))
= Re(Y_2^1(\theta,\phi)) = Re(A\cos\theta\sin\theta e^{i\phi}) = A\cos\theta\sin\theta \cos\phi
$$

$$
A = \frac{-1}{2}\sqrt{\frac{15}{2\pi}}
$$

Analytical expression for U and V:

$$
U(\theta, \phi)=-\frac{1}{r} \frac{\partial \psi}{\partial \phi} = \frac{A\sin\phi}{r}\cos\theta\sin\theta
$$

$$
V(\theta, \phi)=\frac{\sin \theta}{r}\frac{\partial \psi}{\partial \theta} = \frac{A\cos\phi}{r}(\cos^2\theta\sin\theta-\sin^3\theta)
$$

In [2]:
N_t = 100
N_p = 200
#make the starting point very small
theta = np.linspace(1e-6, np.pi, N_t) #100pts 
phi = np.linspace(0, 2*np.pi, N_p)  #200pts
t, p = np.meshgrid(theta, phi) #create a matrix with size theta x phi (100x200), 20000 pts
lMax = 80 #lMax is N
omega = 3.6 #omega is te angular rotation
nu = 1.5  #\nu is kinematic viscosity
T = 4 #total time
r = 1 #let r=1

In [3]:
#let phi = pi/2
sin_p = np.sin(np.pi/2)
cos_p = np.cos(np.pi/2)

In [4]:
A = (-1/2)*((15/(2*np.pi))**(1/2))
psi_tp_a = A*np.cos(t)*np.sin(t)*cos_p #nanalytical psi
psi_tp = np.real(sph_harm (1, 2, p.reshape(-1), t.reshape(-1)).reshape(N_p, N_t)) #numerical psi

In [5]:
#U_tp_n is the numerical expression of U_tp
U_tp_n = -(1/r)*np.gradient(psi_tp, phi, axis=0)
V_tp_n = (np.sin(t)/r)*np.gradient(psi_tp, theta, axis=1)

U_tp = (A/r)*np.cos(t)*np.sin(t)*sin_p #this is obtained by analytical calculation
V_tp = (A*cos_p/r)*(np.cos(t)**2*np.sin(t)-np.sin(t)**3)

psi_nm = projection(psi_tp, theta, phi, lMax)
psi_TP = reconstruction(psi_nm, theta, phi, lMax) #reconstructed


In [6]:
u_nm = projection(U_tp, theta, phi, lMax) #project U_tp to u_nm
v_nm = projection(V_tp, theta, phi, lMax) 

U_TP = reconstruction(u_nm, theta, phi, lMax) #reconstructed U(theta, phi) using u_nm
V_TP = reconstruction(v_nm, theta, phi, lMax) 

In [7]:
u_nm_n = projection(U_tp_n, theta, phi, lMax) #project U_tp to u_nm
v_nm_n = projection(V_tp_n, theta, phi, lMax) 

U_TP_n = reconstruction(u_nm_n, theta, phi, lMax) #reconstructed U(theta, phi) using u_nm
V_TP_n = reconstruction(v_nm_n, theta, phi, lMax) 

In [8]:
np.save('U_tp_80', U_tp)
np.save('u_nm_80', u_nm)
np.save('V_tp_80', V_tp)
np.save('v_nm_80', v_nm)

Analytical expression of different quantities:
$$
\zeta = (\nabla \times \hat{u})_r = \frac{1}{r\sin\theta}(\frac{\partial}{\partial \theta}(u_\phi\sin\theta)-\frac{\partial}{\partial \phi}u_\theta) = \frac{1}{r\sin\theta}(\frac{\partial}{\partial \theta}V-\frac{\partial}{\partial\phi}\frac{U}{\sin\theta})
$$

$$
U'(\theta, \phi)=V\zeta+2\Omega V \cos\theta
$$

$$
V'(\theta, \phi)=-U\zeta-2\Omega U \sin \theta
$$

$$
\nabla^2P=-\frac{\partial U'}{\partial \cos\theta}+\frac{1}{\sin^2\theta}\frac{\partial V'}{\partial \phi} = \frac{1}{\sin\theta}\frac{\partial U'}{\partial \theta}+\frac{1}{\sin^2\theta}\frac{\partial V'}{\partial \phi}
$$

$$
p_{nm}=\frac{-1}{n(n+1)}(\nabla^2P)_{nm}
$$

In [9]:
zeta_nm = np.zeros((lMax, lMax), dtype=complex)
for m in range (1, lMax):
    for n in range (m, lMax):
        zeta_nm[n,m] = -1j*(n*(n+1)/m)*u_nm[n,m]

zeta_tp = reconstruction(zeta_nm, theta, phi, lMax)
U1_tp = V_tp*zeta_tp+2*omega*np.cos(t)*V_tp #U'(theta, phi)
V1_tp = -1*U_tp*zeta_tp-2*omega*np.sin(t)*U_tp #V'(theta, phi)
u1_nm = projection(U1_tp, theta, phi, lMax) #u'_nm
v1_nm = projection(V1_tp, theta, phi, lMax) #v'_nm

In [10]:
#divP = -1*np.gradient(U1_tp, np.cos(theta), axis=1)+(1/(np.sin(t)**2))*np.gradient(V1_tp, phi, axis=0) #\nabla^2 P
divP = (1/np.sin(t))*np.gradient(U1_tp, theta, axis=1)+(1/(np.sin(t)**2))*np.gradient(V1_tp, phi, axis=0) #\nabla^2 P

divP_nm = projection(divP, theta, phi, lMax) #\nabla^2 P_nm
p_nm = np.zeros((lMax, lMax), dtype=complex)
for m in range (1, lMax):
    for n in range (m, lMax):
        p_nm[n,m] = (-1/(n*(n+1)))*divP_nm[n,m]

In [11]:
print(zeta_tp.shape)

(200, 100)


In [12]:
np.save('zeta_nm_80', zeta_nm)
np.save('zeta_tp_80', zeta_tp)
np.save('U1_tp_80', U1_tp)
np.save('u1_nm_80', u1_nm)
np.save('V1_tp_80', V1_tp)
np.save('v1_nm_80', v1_nm)
np.save('divP_80', divP)
np.save('divP_nm_80', divP_nm)
np.save('p_nm_80', p_nm)

In [13]:
# fig,ax = plt.subplots(1,4);
# ax[0].contourf(np.real(zeta_nm))
# ax[0].set_title('zeta_nm')
# ax[1].contourf(np.real(u1_nm))
# ax[1].set_title("u'_nm")
# ax[2].contourf(np.real(v1_nm))
# ax[2].set_title("v'_nm")
# ax[3].contourf(np.real(p_nm))
# ax[3].set_title('p_nm')
# fig.show()

For the above arbitrary stream function $$ \psi(\theta, \phi)= Re(Y_n^m(\theta,\phi))
= Re(Y_2^1(\theta,\phi)) = Re(A\cos\theta\sin\theta e^{i\phi}) = A\cos\theta\sin\theta \cos\phi
$$

The analytical expression of the harmonics quantities is calculated as:

$$
\zeta = \frac{A\cos\phi}{r^2}(-5\cos\theta\sin\theta+\frac{\cos^3\theta}{\sin\theta}-\frac{\cos\theta}{\sin\theta})
$$

$$
U' = \frac{A^2\cos^2\phi}{r^3} (5\cos\theta\sin^4\theta + \cos^5\theta - 6\cos^3\theta\sin^2\theta - \cos^3\theta + \cos\theta\sin^2\theta) + \frac{2\Omega A\cos\phi}{r}(\cos^3\theta\sin\theta - \cos\theta\sin^3\theta)
$$


$$
V' = \frac{-A^2\cos\phi\sin\phi}{r^3} (-5\cos^2\theta\sin^2\theta + \cos^4\theta -\cos^2\theta) + \frac{-2\Omega A\sin\phi}{r}\cos\theta\sin^2\theta
$$

$$
\nabla^2P = \frac{A^2\cos^2\phi}{r^3}(38\cos^2\theta\sin^2\theta - 17\cos^4\theta + 5\cos^2\theta - 5\sin^4\theta - \sin^2\theta) + \frac{2\Omega A\cos\phi}{r}(-6\cos^2\theta\sin\theta + \frac{\cos^4\theta}{\sin\theta} + \sin^3\theta) + \frac{-A^2}{r^3}(\cos^2\phi - \sin^2\phi) (-5\cos^2\theta + \frac{\cos^4\theta}{\sin^2\theta} - \frac{\cos^2\theta}{\sin^2\theta}) + \frac{-2\Omega A \cos\phi}{r}\cos\theta
$$

$$\textit{"_a" denotes the analytical expression obtained by hand (equations above)}$$

$$
\zeta = \frac{A\cos\phi}{r^2}(-5\cos\theta\sin\theta+\frac{\cos^3\theta}{\sin\theta}-\frac{\cos\theta}{\sin\theta})
$$

In [14]:
zeta_tp_a = A*cos_p*(-5*np.cos(t)*np.sin(t)+(np.cos(t)**3/np.sin(t))-(np.cos(t)/np.sin(t)))/(r**2) 

zeta_nm_a = projection(zeta_tp_a, theta, phi, lMax)


In [15]:
print ("max difference between analytical and numerical zeta_tp\n",np.amax(np.abs(zeta_tp_a-zeta_tp)))
print ("max difference between analytical and numerical zeta_nm\n",np.amax(np.abs(zeta_nm_a-zeta_nm)))

#plt.plot(zeta_nm_a[10,:], color='red')
#plt.plot(zeta_nm[10,:], color='green')

max difference between analytical and numerical zeta_tp
 1.5406338484690993
max difference between analytical and numerical zeta_nm
 0.028114577837931365


In [16]:
np.save('zeta_tp_a_80', zeta_tp_a)
np.save('zeta_nm_a_80', zeta_nm_a)

In [17]:
zeta_nm_a_error = np.abs(zeta_nm_a-zeta_nm)/zeta_nm
# plt.plot(zeta_nm_a, zeta_nm_a_error)
# plt.title("zeta_nm_a vs error")
# plt.xlabel("zeta_nm_a")
# plt.ylabel("error")

/tmp/ipykernel_436/2319491453.py:1: RuntimeWarning: divide by zero encountered in divide
  zeta_nm_a_error = np.abs(zeta_nm_a-zeta_nm)/zeta_nm
/tmp/ipykernel_436/2319491453.py:1: RuntimeWarning: invalid value encountered in divide
  zeta_nm_a_error = np.abs(zeta_nm_a-zeta_nm)/zeta_nm


$$
U' = \frac{A^2\cos^2\phi}{r^3} (5\cos\theta\sin^4\theta + \cos^5\theta - 6\cos^3\theta\sin^2\theta - \cos^3\theta + \cos\theta\sin^2\theta) + \frac{2\Omega A\cos\phi}{r}(\cos^3\theta\sin\theta - \cos\theta\sin^3\theta)
$$

In [18]:
a_U = A**2*cos_p**2*(5*np.cos(t)*np.sin(t)**4+np.cos(t)**5-6*np.cos(t)**3*np.sin(t)**2-np.cos(t)**3+np.cos(t)*np.sin(t)**2)/(r**3)
b_U = 2*omega*A*cos_p*(np.cos(t)**3*np.sin(t)-np.cos(t)*np.sin(t)**3)/r

U1_tp_a=a_U+b_U
u1_nm_a = projection(U1_tp_a, theta, phi, lMax)


In [19]:
print ("max difference between analytical and numerical U1_tp\n",np.amax(np.abs(U1_tp_a-U1_tp)))
print ("max difference between analytical and numerical u1_nm\n",np.amax(np.abs(u1_nm_a-u1_nm)))

#plt.plot(u1_nm_a[10,:], color='red')
#plt.plot(u1_nm[10,:], color='green')

max difference between analytical and numerical U1_tp
 4.101490367299765e-17
max difference between analytical and numerical u1_nm
 1.0889825827747403e-18


In [20]:
np.save('U1_tp_a_80', U1_tp_a)
np.save('u1_nm_a_80', u1_nm_a)

In [21]:
U1_tp_a_error = np.abs(U1_tp_a-U1_tp)/U1_tp
# plt.plot(U1_tp_a, U1_tp_a_error)
# plt.title("U1_tp_a vs error")
# plt.xlabel("U1_tp_a")
# plt.ylabel("error")

In [22]:
u1_nm_a_error = np.abs(u1_nm_a-u1_nm)/u1_nm
# plt.plot(u1_nm_a, u1_nm_a_error)
# plt.title("u1_nm_a vs error")
# plt.xlabel("u1_nm_a")
# plt.ylabel("error")

/tmp/ipykernel_436/1498965638.py:1: RuntimeWarning: invalid value encountered in divide
  u1_nm_a_error = np.abs(u1_nm_a-u1_nm)/u1_nm


$$
V' = \frac{-A^2\cos\phi\sin\phi}{r^3} (-5\cos^2\theta\sin^2\theta + \cos^4\theta -\cos^2\theta) + \frac{-2\Omega A\sin\phi}{r}\cos\theta\sin^2\theta
$$

In [23]:
a_V = -1*A**2*cos_p*sin_p*(-5*np.cos(t)**2*np.sin(t)**2+np.cos(t)**4-np.cos(t)**2)/(r**3)
b_V = -2*omega*A*sin_p*np.cos(t)*np.sin(t)**2/r

V1_tp_a = a_V+b_V
v1_nm_a = projection(V1_tp_a, theta, phi, lMax)


In [24]:
print ("max difference between analytical and numerical V1_tp\n",np.amax(np.abs(V1_tp_a-V1_tp)))
print ("max difference between analytical and numerical v1_nm\n",np.amax(np.abs(v1_nm_a-v1_nm)))
 
#plt.plot(v1_nm_a[10,:], color='red')
#plt.plot(v1_nm[10,:], color='green')

max difference between analytical and numerical V1_tp
 0.5647457134800993
max difference between analytical and numerical v1_nm
 0.009911966969119683


In [25]:
np.save('V1_tp_a_80', V1_tp_a)
np.save('v1_nm_a_80', v1_nm_a)

In [26]:
V1_tp_a_error = np.abs(V1_tp_a-V1_tp)/V1_tp
# plt.plot(V1_tp_a, V1_tp_a_error)
# plt.title("V1_tp_a vs error")
# plt.xlabel("V1_tp_a")
# plt.ylabel("error")

In [27]:
v1_nm_a_error = np.abs(v1_nm_a-v1_nm)/v1_nm
# plt.plot(v1_nm_a, v1_nm_a_error)
# plt.title("v1_nm_a vs error")
# plt.xlabel("v1_nm_a")
# plt.ylabel("error")

/tmp/ipykernel_436/4272200411.py:1: RuntimeWarning: invalid value encountered in divide
  v1_nm_a_error = np.abs(v1_nm_a-v1_nm)/v1_nm


$$
\nabla^2P = \frac{A^2\cos^2\phi}{r^3}(38\cos^2\theta\sin^2\theta - 17\cos^4\theta + 5\cos^2\theta - 5\sin^4\theta - \sin^2\theta) + \frac{2\Omega A\cos\phi}{r}(-6\cos^2\theta\sin\theta + \frac{\cos^4\theta}{\sin\theta} + \sin^3\theta) + \frac{-A^2}{r^3}(\cos^2\phi - \sin^2\phi) (-5\cos^2\theta + \frac{\cos^4\theta}{\sin^2\theta} - \frac{\cos^2\theta}{\sin^2\theta}) + \frac{-2\Omega A \cos\phi}{r}\cos\theta
$$

In [28]:
a_divP=A**2*cos_p**2*(38*np.cos(t)**2*np.sin(t)**2-17*np.cos(t)**4+5*np.cos(t)**2-5*np.cos(t)**4-np.sin(t)**2)/(r**3)
b_divP=2*omega*A*cos_p*(-6*np.cos(t)**2*np.sin(t)+(np.cos(t)**4/np.sin(t))+np.sin(t)**3)/r
c_divP=-1*A**2*(cos_p**2-sin_p**2)*(-5*np.cos(t)**2+(np.cos(t)**4/(np.sin(t)**2))-(np.cos(t)**2/(np.sin(t)**2)))/(r**3)
d_divP=-2*omega*A*cos_p*np.cos(t)/r

divP_a=a_divP+b_divP+c_divP+d_divP
divP_nm_a = projection(divP_a, theta, phi, lMax)


In [29]:
print ("max difference between analytical and numerical divP\n",np.amax(np.abs(divP_a-divP)/divP_a))
print ("max difference between analytical and numerical divP_nm\n",np.amax(np.abs(divP_nm_a-divP_nm)))

#plt.plot(divP_nm_a[30,:], color='red')
#plt.plot(divP_nm[30,:], color='green')

max difference between analytical and numerical divP
 -0.0019393980155619922
max difference between analytical and numerical divP_nm
 4.189044903334596


In [30]:
np.save('divP_a_80', divP_a)
np.save('divP_nm_a_80', divP_nm_a)

In [31]:
divP_a_error = np.abs(divP_a-divP)/divP
# plt.plot(divP_a, divP_a_error)
# plt.title("divP_a vs error")
# plt.xlabel("divP_a")
# plt.ylabel("error")

In [32]:
divP_nm_a_error = np.abs(divP_nm_a-divP_nm)/divP_nm
# plt.plot(divP_nm_a, divP_nm_a_error)
# plt.title("divP_nm_a vs error")
# plt.xlabel("divP_nm_a")
# plt.ylabel("error")

/tmp/ipykernel_436/2667720334.py:1: RuntimeWarning: invalid value encountered in divide
  divP_nm_a_error = np.abs(divP_nm_a-divP_nm)/divP_nm
